In [1]:
from pathlib import Path
from omnibelt import toposort, load_json, save_json, pformat, pformat_vars
from itertools import product, combinations
# import pyperclip
import random
from PIL import Image
from graphviz import Digraph
import matplotlib.pyplot as plt
import io
# import clipboard
import networkx as nx
from tqdm.notebook import tqdm
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from omniply import tool, ToolKit, Context

import numpy as np
import torch
from causalbenchmark.novo.simulation.models import Bernoulli, ConditionalBernoulli, BernoulliNetwork
from causalbenchmark.novo.seeding import NewsHeadlines, StatisticsPrompting, Story
# from causalbenchmark.novo.seeding.prompting import GraphInfo
from causalbenchmark.novo.templating import FixedTemplate, SimpleTemplater
# from causalbenchmark.novo.verbalization import misc
from causalbenchmark.novo import misc
from causalbenchmark.novo.verbalization.decision import Decision
from causalbenchmark.novo.templating import Template
from causalbenchmark.novo.verbalization.clause import ClauseVerbalization

In [2]:
story_names = [
	'test19',
	'test20',
	'test21',
]
stories = []
for name in story_names:
	story = Story(story_root=misc.data_root() / 'stories', story_id=name)#.populate_defaults()
	stories.append(story)
	
	story['stats'] = {int(k): v for k, v in story['stats'].items()}
	story['verbs'] = {k: {int(k2): v2 for k2, v2 in v.items()} for k, v in story['verbs'].items()}
	for k, v in story['verbs'].items():
		for k2, v2 in v.items():
			if 'value' in v2:
				v2['value'] = {int(k3): v3 for k3, v3 in v2['value'].items()}
	story['questions']['questions'] = {int(k): v for k, v in story['questions']['questions'].items()}
len(stories)

In [3]:
def story_atoms(story):
	varverbs = {}
	for var, raw in story['verbs'].items():
		info = {'values': {0: {}, 1: {}}}
		
		info['variable'] = raw[1]['variable']
		info['subject'] = raw[2]['subject']
		info['domain'] = raw[4]['domain']
		
		for i in [0, 1]:
			info_val = info['values'][i]
			info_val['verb'] = raw[2]['value'][i] # goes with subject
			info_val['phrase'] = raw[3]['value'][i]
			info_val['descriptor'] = raw[4]['value'][i] # goes with domain
			info_val['event'] = raw[5]['value'][i]
			info_val['conditional'] = raw[6]['value'][i]
			info_val['interventional'] = raw[7]['value'][i]
			
		varverbs[var] = info
	
	story['sheet'] = varverbs
	return story

for story in stories:
	story_atoms(story)

In [4]:
story = random.choice(stories)
rawverbs = story['verbs']
rawvars = {node['name']: node for node in story['nodes']}
# next(iter(rawverbs.values()))

In [5]:
varname = random.choice(list(rawverbs.keys()))
nodeinfo = [node for node in story['nodes'] if node['name'] == varname][0]
varinfo = story['sheet'][varname]
varval = 0
varname, nodeinfo['values'][varval]

('StadiumDecision', 'No Construction')

In [6]:
info = {**varinfo, **varinfo['values'][0]}
del info['values']
info.keys()

dict_keys(['variable', 'subject', 'domain', 'verb', 'phrase', 'descriptor', 'event', 'conditional', 'interventional'])

In [24]:
precise_templates = [
	['there', 'is', 'a', '{quantity}', '{prob_word}', 'that', '{position}'],
	['the', '{chance_word}', 'that', '{position}', '{"are" if chance_word == "odds" else "is"}', '{quantity}'],
	['{quantity}', 'of', 'the', 'time', '{position}'],
	['{quantity}', 'of', '{domain}', '{descriptor}'],
	['with a', '{confidence_word}', 'of {quantity}, {position}']
]

position_templates = [
	['{subject}', '{verb}',],
	['{domain}', '{descriptor}'],
	# ['{descriptor}'],
	['{phrase}',],
	['{event}',],
]

def sample_text(ident=None, **static):
	if ident is None:
		ident = {}
	story_id = ident.setdefault('story_id', random.randint(0,len(stories)-1))
	story = stories[story_id]
	
	var_id = ident.setdefault('var_id', random.randint(0,len(story['nodes'])-1))
	varname = story['nodes'][var_id]['name']
	varinfo = story['sheet'][varname]
	varval = ident.setdefault('value_id', random.choice([0, 1]))
	
	info = {**varinfo, **varinfo['values'][varval]}
	del info['values']
	
	ctx = ClauseVerbalization().populate_variable_info(info, **static).populate_default(precise_templates=precise_templates, position_templates=position_templates)
	
	ctx.include(Decision(['confidence', 'certainty'], 'confidence_word'))
	
	ctx.update(ident)
	return ctx

In [50]:
samples = []
for _ in range(30):
	ident = {
		'story_id': 0,
		'var_id': 0,
		'value_id': 1,
		
		# 'clause_id': 0,
		# 'position_id': 3,
		
	}
	ctx = sample_text(ident, mean=0.38)
	text = Template.detok(ctx['clause'], capitalize=True, sentence=True)
	ident.update(ctx.identity())
	samples.append({'text': text, 'ctx': ctx, 'ident': ident})

cols = []
for row in samples:
	for key in row['ident']:
		if key not in cols:
			cols.append(key)

table = [(i, 
		  row['text'], 
		  # row['ctx']['event'], row['ctx']['event'], row['ctx']['phrase'], row['ctx']['subject'], row['ctx']['verb'],  
		  # *[row['ident'].get(key, '-') for key in cols]
		  ) for i, row in enumerate(samples)]
print(tabulate(table, headers=['i', 'text', *cols]))# *list(map(str, range(len(cols))))]))

  i  text
---  --------------------------------------------------------------------------
  0  With a confidence of 38%, the refinery has been operating for a long time.
  1  38% of the time the refinery has been operating for many years.
  2  The chance that the refinery has many years of operation is 38%.
  3  38% of refineries have been operating for many years.
  4  With a certainty of 38%, the refinery has been operating for a long time.
  5  There is a 38% chance that the refinery is an old establishment.
  6  38% of refineries have been operating for many years.
  7  There is a 38% chance that the refinery is an older establishment.
  8  The likelihood that the refinery has been operating for many years is 38%.
  9  38% of refineries have been operating for many years.
 10  38% of the time the refinery is quite old.
 11  38% of the time the refinery is an old establishment.
 12  There is a 38% chance that the refinery is an older establishment.
 13  With a confidence of 38%, the

In [78]:
sample = samples[0]
ident = sample['ident']
ctx = sample['ctx']
text = sample['text']

print(text)
print(ident)

The odds that this incident got limited coverage in the news are 62%.
{'story_id': 0, 'var_id': 6, 'value_id': 0, 'verb_id': 2, 'clause_id': 1, 'position_id': 0, 'chance_word_id': 3}


In [ ]:
ident = {}
ctx = sample_text(ident, mean=0.57)



text = Template.detok(ctx['clause'], capitalize=True, sentence=True)
ident.update(ctx.identity())
print(text)
ident

In [33]:

selection = {}




story = random.choice(stories)
varname = random.choice(list(rawverbs.keys()))
varinfo = story['sheet'][varname]
varval = 0

ctx = ClauseVerbalization().populate_variable_info(info, mean=0.57).populate_default(precise_templates=precise_templates, position_templates=position_templates)

# ctx['clause_id'] = 3
ctx['position_id'] = 0

text = Template.detok(ctx['clause'], capitalize=True, sentence=True)
print(text)
ctx.identity()

The chance that this incident was scarcely covered by the media is 57%.


{'verb_id': 1, 'clause_id': 1, 'position_id': 0, 'chance_word_id': 1}

In [10]:
# ClauseVerbalization._precise_templates[0]

In [8]:

prob_words = ['probability', 'chance', 'likelihood']
chance_words = ['probability', 'chance', 'likelihood', 'odds']



In [18]:
def detok(tokens):
	toks = [', ' if tok == ',' else tok for tok in tokens]
	sentence = ' '.join(toks) + '.'
	return sentence.capitalize()

In [ ]:
# 1. One of the variables is {descriptor}.
# 2. {subject} [often/usually/sometimes/rarely/etc.] {predicate}.
# 3. There is a [number]% chance that {noun-clause}.
# 4. [In/For/With/etc.] [number]% of {domain}, there {phrase}.
# 5. Conditional sentence: {condition}, [some consequence].
# 6. Interventional: If {action}, [some effect].

In [19]:
# The man that she loved went home.
# She realized that he loves her.

NameError: name 'AbstractGadget' is not defined

In [ ]:
# 1. One of the variables is {variable}.
# 2. We estimate {subject} [often/usually/sometimes/rarely/etc.] {value}.
# 3. There is a [number]% chance that {value}.
# 4. [number]% of {domain} {value}.
# 5. [number]% of the time {value}.
# 6. Conditional sentence: {value}, [some consequence].
# 7. Interventional: If {value}, [some effect].